In [4]:
params = {
    "all_wheels_on_track": bool,
    "closest_waypoints": [int, int],
    "distance_from_center": float,
    "is_crashed": bool,
    "is_left_of_center": bool,
    "is_offtrack": bool,
    "heading": float,
    "progress": float,
    "speed": float,
    "steering_angle": float,
    "steps": int,
    "track_length": float,
    "track_width": float,
    "waypoints": [(float, float),]
}


In [40]:
import math
import numpy as np
import pandas as pd
waypoints = np.load('reinvent_base.npy')
waypoints[:, 2:4].shape

[[5.15626907 1.06500494]
 [5.30604792 1.06505001]
 [5.45556688 1.065117  ]
 [5.60494995 1.06533599]
 [5.75636387 1.06528699]
 [5.90788317 1.064587  ]
 [6.05638313 1.06333899]
 [6.18435717 1.06661201]
 [6.3118062  1.07716703]
 [6.43282795 1.09338105]
 [6.47744083 1.11110198]
 [6.57148504 1.16342795]
 [6.67055082 1.23154294]
 [6.72773123 1.28162706]]


In [ ]:
import math
import pandas as pd
import numpy as np

def get_distance_from_borders(track_width, distance_from_center, is_left_of_center):
    if is_left_of_center:
        distance_from_right = (track_width / 2) + distance_from_center
        distance_from_left = track_width - distance_from_right
        return (distance_from_right, distance_from_left)
    elif not is_left_of_center:
        distance_from_left = (track_width / 2) + distance_from_center
        distance_from_right = track_width - distance_from_left
        return (distance_from_right, distance_from_left)
    else:
        raise Exception("Something went wrong with the get_distance_from_borders function")
def get_next_waypoints(waypoints, current_closest_waypoint, num_waypoints=15):
    waypoints_slice = waypoints[:, 2:4]
    first_idx = 0
    for idx, row in enumerate(waypoints_slice):
        if np.array_equal(row, current_closest_waypoint):
            first_idx = idx
    return waypoints_slice[first_idx + 1:first_idx+num_waypoints]

def reward_function(params):

    straights = {113: (2.306056022644043, 1.0518460273742676), 114: (2.445446014404297, 1.056846022605896), 115: (2.6014020442962646, 1.0617640018463135), 116: (2.7572851181030273, 1.064208984375), 117: (2.913090944290161, 1.0641800165176392), 118:(3.0593700408935547, 1.0636550188064575), 0: (3.0593700408935547, 1.0636550188064575), 1: (3.20845890045166, 1.0641330480575562), 2: (3.359158992767334, 1.064363956451416), 3: (3.50888991355896, 1.0644019842147827), 4: (3.6586480140686035, 1.0644609928131104), 5: (3.808418035507202, 1.0645170211791992), 6: (3.958184003829956, 1.0645689964294434), 7: (4.1079421043396, 1.0646209716796875), 8: (4.257699012756348, 1.0646740198135376), 9: (4.4074578285217285, 1.064728021621704), 10: (4.557218074798584, 1.0647809505462646), 11: (4.706972122192383, 1.064836025238037), 12: (4.856725215911865, 1.0648939609527588), 13: (5.006490230560303, 1.06495201587677), 14: (5.156269073486328, 1.0650049448013306), 15: (5.3060479164123535, 1.0650500059127808), 16: (5.455566883087158, 1.0651170015335083), 17: (5.604949951171875, 1.065335988998413), 18: (5.756363868713379, 1.0652869939804077), 19: (5.907883167266846, 1.0645869970321655), 20: (6.05638313293457, 1.0633389949798584), 21: (6.184357166290283, 1.0666120052337646), 45: (5.933832168579102, 2.3911380767822266), 46: (5.782729148864746, 2.3898088932037354), 47: (5.632474899291992, 2.388612985610962), 48: (5.480686187744141, 2.3880488872528076), 49: (5.335058212280273, 2.387484073638916), 50: (5.191987991333008, 2.3845291137695312), 59: (3.7711479663848873, 3.162066936492921), 60: (3.6768300533294678, 3.277514934539795), 61: (3.5819129943847656, 3.3939499855041504), 62: (3.4873900413513184, 3.509998083114624), 63: (3.392906904220581, 3.625926971435547), 64: (3.2982048988342285, 3.741957902908325), 65: (3.2022480964660645, 3.8603339195251465), 66: (3.1152100563049316, 3.96835994720459), 73: (2.436302900314331, 4.117131233215332), 74: (2.3038649559020996, 4.112199783325195), 75: (2.1552109718322754, 4.107230186462402), 76: (2.0027310848236084, 4.103099822998047), 77: (1.8536850214004517, 4.0990309715271), 78: (1.7066760063171387, 4.094202995300293), 79: (1.5570969581604004, 4.088045120239258), 91: (1.137742042541504, 3.236654043197632), 92: (1.1811360120773315, 3.0822010040283203), 93: (1.220324993133545, 2.9256539344787598), 94: (1.254794955253601, 2.7688379287719727), 95: (1.2850170135498047, 2.622812032699585), 96: (1.3154289722442627, 2.4760611057281494), 97: (1.3458080291748047, 2.3294270038604736), 98: (1.3761910200119019, 2.1827940940856934), 99: (1.406581997871399, 2.0361480712890625), 100: (1.4369670152664185, 1.8893100023269653), 101: (1.4672410488128662, 1.7433960437774658), 102: (1.4979009628295898, 1.597417950630188), 103: (1.5273100137710571, 1.461290955543518)}
    left_turns = {22: (6.3118062019348145, 1.07716703414917), 23: (6.432827949523926, 1.0933810472488403), 24: (6.47744083404541, 1.1111019849777222), 25: (6.5714850425720215, 1.1634279489517212), 26: (6.67055082321167, 1.231542944908142), 27: (6.727731227874756, 1.2816270589828491), 28: (6.778569221496582, 1.351040005683899), 29: (6.829151153564453, 1.4422310590744019), 30: (6.870870113372803, 1.543166995048523), 31: (6.898098945617676, 1.636909008026123), 32: (6.9029860496521, 1.7000770568847656), 33: (6.889245986938477, 1.805264949798584), 34: (6.866875171661377, 1.9087029695510864), 35: (6.8341288566589355, 2.004347085952759), 36: (6.793342113494873, 2.081338882446289), 37: (6.737051010131836, 2.1548550128936768), 38: (6.664588928222656, 2.2276480197906494), 39: (6.588120937347412, 2.2869949340820312), 40: (6.508316993713379, 2.3313000202178955), 41: (6.421780109405518, 2.363363027572632), 42: (6.332312107086182, 2.3817288875579834), 43: (6.214798927307129, 2.389333963394165), 44: (6.077500820159912, 2.392366886138916), 67: (3.0651309490203857, 4.0154709815979), 68: (2.985136032104492, 4.064317226409912), 69: (2.8928120136260986, 4.109425067901611), 70: (2.8303000926971436, 4.118078231811523), 71: (2.8071720600128174, 4.116944789886475), 72: (2.627346992492676, 4.116711139678955), 80: (1.4433189630508423, 4.08112907409668), 81: (1.3885200023651123, 4.066157817840576), 82: (1.3314000368118286, 4.033370018005371), 83: (1.2584710121154785, 3.973881959915161), 84: (1.190619945526123, 3.8995161056518555), 85: (1.1391240358352661, 3.8233230113983154), 86: (1.1083790063858032, 3.7523629665374756), 87: (1.0925209522247314, 3.6791059970855713), 88: (1.088243007659912, 3.5748629570007324), 89: (1.0941170454025269, 3.4608349800109863), 90: (1.1105600595474243, 3.348402976989746), 104: (1.5593780279159546, 1.350823998451233), 105: (1.6024500131607056, 1.2406150102615356), 106: (1.6249639987945557, 1.2196669578552246), 107: (1.688094973564148, 1.145346999168396), 108: (1.7174400091171265, 1.1385610103607178), 109: (1.8113280534744263, 1.1070239543914795), 110: (1.9283230304718018, 1.0826369524002075), 111: (2.0541889667510986, 1.0645229816436768), 112: (2.1800119876861572, 1.0542750358581543)}
    right_turns = {51: (4.99628210067749, 2.3867130279541016), 52: (4.814363956451416, 2.4097490310668945), 53: (4.619063854217529, 2.458385944366455), 54: (4.420148849487305, 2.541192054748535), 55: (4.2387518882751465, 2.6570489406585693), 56: (4.089457035064697, 2.7907869815826416), 57: (3.969646930694581, 2.9185070991516104), 58: (3.8614220619201656, 3.0494639873504648)}

    # Initialize params
    closest_waypoints = params['closest_waypoints']
    x_loc = params['x']
    y_loc = params['y']
    distance_from_center = params['distance_from_center']
    is_left_of_center = params['is_left_of_center']
    heading = params['heading']
    progress = params['progress']
    speed = params['speed']
    steering_angle = params['steering_angle']
    steps = params['steps']
    track_width = params['track_width']
    waypoints = params['waypoints']
    waypoints_slice = waypoints[:, 2:4]

    waypoint_ahead = waypoints[closest_waypoints[1]]
    waypoint_behind = waypoints[closest_waypoints[0]]
    
    # Initialize the typical reward
    reward = 1.0

    next_waypoints = get_next_waypoints(waypoints_slice, waypoint_ahead)
    distance_from_borders = get_distance_from_borders(track_width, distance_from_center, is_left_of_center)
    
    if 

    return float(reward)